In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

21/12/18 11:51:44 WARN Utils: Your hostname, chaitanya-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/12/18 11:51:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/18 11:51:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_train = spark.read.csv('training.csv',inferSchema=True,header=False)
df_test = spark.read.csv('testdata.csv',inferSchema=True,header=False)

In [4]:
df_train.show(5)

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 5 rows



In [8]:
df_test = df_test.filter(df_test._c0 != 2)
df_test.show()

+---+---+--------------------+-------+--------------+--------------------+
|_c0|_c1|                 _c2|    _c3|           _c4|                 _c5|
+---+---+--------------------+-------+--------------+--------------------+
|  4|  3|Mon May 11 03:17:...|kindle2|        tpryan|@stellargirl I lo...|
|  4|  4|Mon May 11 03:18:...|kindle2|        vcu451|Reading my kindle...|
|  4|  5|Mon May 11 03:18:...|kindle2|        chadfu|Ok, first assesme...|
|  4|  6|Mon May 11 03:19:...|kindle2|         SIX15|@kenburbary You'l...|
|  4|  7|Mon May 11 03:21:...|kindle2|      yamarama|@mikefish  Fair e...|
|  4|  8|Mon May 11 03:22:...|kindle2|  GeorgeVHulme|@richardebaker no...|
|  0|  9|Mon May 11 03:22:...|    aig|       Seth937|Fuck this economy...|
|  4| 10|Mon May 11 03:26:...| jquery|     dcostalis|Jquery is my new ...|
|  4| 11|Mon May 11 03:27:...|twitter|       PJ_King|       Loves twitter|
|  4| 12|Mon May 11 03:29:...|  obama|   mandanicole|how can you not l...|
|  0| 14|Mon May 11 03:32

In [9]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

In [10]:
regexTokenizer = RegexTokenizer(inputCol="_c5", outputCol="tokens", pattern="\\W")
add_stopwords = ["http","https","amp","rt","t","c","the","@"]
stopwordsRemover = StopWordsRemover(inputCol="tokens", outputCol="clean_tokens").setStopWords(add_stopwords)
countVectors = CountVectorizer(inputCol="clean_tokens", outputCol="features", vocabSize=10000, minDF=5)
label_stringIdx = StringIndexer(inputCol = "_c0", outputCol = "label")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, seed=100)
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx, rf])

In [11]:
pipelineFit = pipeline.fit(df_train)
predictions = pipelineFit.transform(df_test)

21/12/18 12:06:25 WARN MemoryStore: Not enough space to cache rdd_70_1 in memory! (computed 85.0 MiB so far)
21/12/18 12:06:25 WARN BlockManager: Persisting block rdd_70_1 to disk instead.
21/12/18 12:06:25 WARN MemoryStore: Not enough space to cache rdd_70_0 in memory! (computed 85.0 MiB so far)
21/12/18 12:06:25 WARN BlockManager: Persisting block rdd_70_0 to disk instead.
21/12/18 12:06:26 WARN MemoryStore: Not enough space to cache rdd_70_2 in memory! (computed 196.3 MiB so far)
21/12/18 12:06:26 WARN BlockManager: Persisting block rdd_70_2 to disk instead.
21/12/18 12:06:27 WARN MemoryStore: Not enough space to cache rdd_70_3 in memory! (computed 196.3 MiB so far)
21/12/18 12:06:27 WARN BlockManager: Persisting block rdd_70_3 to disk instead.
21/12/18 12:10:07 WARN MemoryStore: Not enough space to cache rdd_70_3 in memory! (computed 294.5 MiB so far)
21/12/18 12:10:27 WARN MemoryStore: Not enough space to cache rdd_70_2 in memory! (computed 294.5 MiB so far)
21/12/18 12:10:46 WARN

In [12]:
predictions.show(50)

+---+---+--------------------+----------------+--------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|_c0|_c1|                 _c2|             _c3|           _c4|                 _c5|               words|            filtered|            features|label|       rawPrediction|         probability|prediction|
+---+---+--------------------+----------------+--------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|  4|  3|Mon May 11 03:17:...|         kindle2|        tpryan|@stellargirl I lo...|[stellargirl, i, ...|[stellargirl, i, ...|(10000,[0,3,7,8,1...|  1.0|[50.7100424980250...|[0.50710042498025...|       0.0|
|  4|  4|Mon May 11 03:18:...|         kindle2|        vcu451|Reading my kindle...|[reading, my, kin...|[reading, my, kin...|(10000,[3,4,7,28,...|  1.0|[47.7941223049679...|[0.

In [13]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  124|
|       1.0|  235|
+----------+-----+



In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [15]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='label',metricName='accuracy').evaluate(predictions)
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 71%


In [16]:
rf_precision=MulticlassClassificationEvaluator(labelCol='label',metricName='weightedPrecision').evaluate(predictions)
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 73%


In [17]:
rf_auc=BinaryClassificationEvaluator(labelCol='label').evaluate(predictions)
print(rf_auc)

0.7818495064257794


In [21]:
pipelineFit.save("./Sentiment_Analysis_model")